# Basic Analysis
Slicing, Dicing, and Graphing.  Simple exploration of the data.

In [15]:
import pandas as pd
data = pd.read_csv('../raw_data/roxy_log.201607211200.csv', encoding='iso-8859-1')
data.head()

,Time (sec),A/F Sens 1 Ratio (AFR),Boost (PSI),Calculated Load (g/rev),Feedback Knock (°),Fine Knock Learn (°),Gear Position (Gear),Ignition Timing (°),MAF (g/s),Primary Ign. (°),RPM (RPM),TD Burst (%),TD Continuous (%),Target Boost (PSI),Throttle Pos. (%),Wastegate Duty (%),Wastegate Max (%),AP Info:[AP3-SUB-001 v1.7.2.0-11034][USDM 2004 Impreza WRX][Reflash: EXPERIMENTAL-TP-cobb2.ptm - Realtime: EXPERIMENTAL-TP-cobb2.ptm]
0,0.000,14.59,-7.19,0.48,0.0,0.0,1,41.5,27.15,41.88,3352,0.0,4.02,-5.12,13,34.12,34.08,0
1,0.004,14.70,-7.19,0.48,0.0,0.0,1,41.5,27.46,41.88,3398,0.0,4.25,-4.95,13,34.12,34.29,0
2,0.014,14.59,-7.19,0.49,0.0,0.0,1,41.5,27.77,41.88,3407,0.0,4.44,-4.87,13,34.12,34.46,0
3,0.016,14.59,-7.14,0.49,0.0,0.0,1,41.5,28.18,42.23,3436,0.0,5.00,-4.68,13,34.51,34.73,0
4,0.019,14.59,-7.08,0.50,0.0,0.0,1,41.5,29.82,42.23,3503,0.0,5.00,-4.56,14,34.51,34.86,0


In [16]:
# 
data.describe()

,Time (sec),A/F Sens 1 Ratio (AFR),Boost (PSI),Calculated Load (g/rev),Feedback Knock (°),Fine Knock Learn (°),Gear Position (Gear),Ignition Timing (°),MAF (g/s),Primary Ign. (°),RPM (RPM),TD Burst (%),TD Continuous (%),Target Boost (PSI),Throttle Pos. (%),Wastegate Duty (%),Wastegate Max (%),AP Info:[AP3-SUB-001 v1.7.2.0-11034][USDM 2004 Impreza WRX][Reflash: EXPERIMENTAL-TP-cobb2.ptm - Realtime: EXPERIMENTAL-TP-cobb2.ptm]
count,19756.000000,19756.000000,19756.000000,19756.000000,19756.000000,19756.000000,19756.000000,19756.000000,19756.000000,19756.000000,19756.000000,19756.000000,19756.000000,19756.000000,19756.000000,19756.000000,19756.000000,19756.0
mean,721.665294,15.538034,-7.721658,0.443494,-0.000854,-0.042994,3.521715,22.365079,18.249331,22.643487,1971.443106,-0.003020,4.361465,-4.127713,7.507795,8.866477,12.057363,0.0
std,416.859554,3.233380,4.825361,0.406604,0.042452,0.241604,2.106500,11.021122,30.369333,11.718651,1196.140508,0.182049,1.469534,4.318753,16.144262,14.149493,15.502971,0.0
min,0.000000,11.250000,-12.300000,0.030000,-2.110000,-2.110000,1.000000,-11.000000,1.560000,5.310000,667.000000,-5.030000,-5.030000,-8.160000,0.000000,0.000000,0.000000,0.0
25%,360.653500,14.360000,-9.980000,0.270000,0.000000,0.000000,1.000000,11.500000,3.920000,11.990000,733.000000,0.000000,5.000000,-5.340000,0.000000,0.000000,0.000000,0.0
50%,721.811500,14.590000,-9.130000,0.310000,0.000000,0.000000,4.000000,20.000000,5.470000,16.910000,1737.500000,0.000000,5.000000,-5.340000,1.000000,0.000000,0.000000,0.0
75%,1082.642250,14.700000,-7.660000,0.460000,0.000000,0.000000,6.000000,34.000000,19.430000,35.550000,3017.000000,0.000000,5.000000,-5.340000,10.000000,21.180000,31.660000,0.0
max,1443.499000,25.610000,18.600000,2.740000,0.000000,0.000000,6.000000,42.500000,231.480000,42.580000,6880.000000,4.810000,5.000000,17.980000,100.000000,45.490000,45.660000,0.0


In [17]:
# slice out just the boost column
boost = data['Boost (PSI)']
boost.head()

0   -7.19
1   -7.19
2   -7.19
3   -7.14
4   -7.08
Name: Boost (PSI), dtype: float64

In [18]:
boost.describe()

count    19756.000000
mean        -7.721658
std          4.825361
min        -12.300000
25%         -9.980000
50%         -9.130000
75%         -7.660000
max         18.600000
Name: Boost (PSI), dtype: float64

In [19]:
# get just boost above 10psi
boost[boost > 10].count()


435

In [27]:
# look for learned knock events
knock = data['Fine Knock Learn (°)']
data[knock < 0].describe()

,Time (sec),A/F Sens 1 Ratio (AFR),Boost (PSI),Calculated Load (g/rev),Feedback Knock (°),Fine Knock Learn (°),Gear Position (Gear),Ignition Timing (°),MAF (g/s),Primary Ign. (°),RPM (RPM),TD Burst (%),TD Continuous (%),Target Boost (PSI),Throttle Pos. (%),Wastegate Duty (%),Wastegate Max (%),AP Info:[AP3-SUB-001 v1.7.2.0-11034][USDM 2004 Impreza WRX][Reflash: EXPERIMENTAL-TP-cobb2.ptm - Realtime: EXPERIMENTAL-TP-cobb2.ptm]
count,665.000000,665.000000,665.000000,665.000000,665.000000,665.000000,665.000000,665.000000,665.000000,665.000000,665.000000,665.000000,665.000000,665.000000,665.000000,665.000000,665.000000,665.0
mean,459.645389,12.631429,6.940496,1.685203,-0.015865,-1.277263,3.150376,19.912030,112.654015,17.947038,3658.215038,-0.028767,2.221699,8.563068,53.234586,28.900571,30.191579,0.0
std,203.965833,1.686369,7.423979,0.646754,0.182408,0.397252,1.522404,7.682626,68.340908,9.967908,1212.798956,0.344551,2.216265,7.430908,35.475840,10.378881,9.814778,0.0
min,0.765000,11.250000,-3.690000,0.900000,-2.110000,-2.110000,1.000000,5.500000,28.900000,6.020000,1751.000000,-5.030000,-5.030000,-7.930000,0.000000,0.000000,0.630000,0.0
25%,315.579000,11.250000,-0.230000,1.050000,0.000000,-1.760000,2.000000,13.000000,42.350000,9.880000,2334.000000,0.000000,0.660000,1.990000,24.000000,22.750000,24.760000,0.0
50%,393.004000,11.370000,5.220000,1.510000,0.000000,-1.050000,3.000000,18.000000,106.220000,14.100000,3996.000000,0.000000,1.970000,6.880000,35.000000,29.020000,30.210000,0.0
75%,623.755000,14.130000,14.480000,2.360000,0.000000,-1.050000,4.000000,27.000000,193.090000,27.110000,4515.000000,0.000000,5.000000,17.850000,100.000000,36.080000,37.340000,0.0
max,1050.851000,22.390000,18.600000,2.740000,0.000000,-0.700000,6.000000,40.000000,231.480000,40.470000,6769.000000,1.230000,5.000000,17.980000,100.000000,45.490000,45.660000,0.0
